In [59]:
import google.cloud.bigquery as bq
import pandas as pd
from keplergl import KeplerGl

In [2]:
%load_ext google.cloud.bigquery

In [3]:
%%bigquery stations

SELECT
stat.id,
LEFT(stat.id, 2) AS country,
stat.state,
stat.name,
stat.latitude,
stat.longitude,
inv.element,
inv.firstyear,
inv.lastyear 
FROM `bigquery-public-data.ghcn_d.ghcnd_stations` stat
LEFT JOIN `bigquery-public-data.ghcn_d.ghcnd_inventory` inv ON stat.id = inv.id
  AND inv.element IN ("TMAX", "TMIN", "ACMC", "AWDR", "AWND", "EVAP", "MXPN")
WHERE
ST_CONTAINS(
  ST_MAKEPOLYGON(
    ST_MAKELINE(
      ARRAY(
              select ST_GEOGPOINT(-12.950869, 53.431059) union all
              select ST_GEOGPOINT(-10.342842, 35.002107) union all
              select ST_GEOGPOINT(-19.931662, 13.986127) union all
              select ST_GEOGPOINT(-9.352620, 0.0)        union all
              select ST_GEOGPOINT(-49.101915, 0)         union all
              select ST_GEOGPOINT(-71.903541, 13.516013) union all
              select ST_GEOGPOINT(-76.957330, 34.883146)
            )
          )
        )
  ,ST_GEOGPOINT(stat.longitude, stat.latitude)) IS TRUE
ORDER BY stat.id

In [4]:
stations.head()

,id,country,state,name,latitude,longitude,element,firstyear,lastyear
0,ACW00011647,AC,,ST JOHNS,17.1333,-61.7833,TMAX,1961.0,1961.0
1,ACW00011647,AC,,ST JOHNS,17.1333,-61.7833,TMIN,1961.0,1961.0
2,BB000078954,BB,,GRANTLEY ADAMS,13.0670,-59.4830,TMAX,1944.0,2018.0
3,BB000078954,BB,,GRANTLEY ADAMS,13.0670,-59.4830,TMIN,1944.0,2018.0
4,BDM00078016,BD,,L F WADE INTL AP KINDLEY FLD,32.3667,-64.6833,TMAX,1949.0,2020.0


In [31]:
n_stations = len(stations.id.unique())
n_coountries = len(stations.country.unique())

print(f'There are {n_coountries} countries and {n_stations} stations within the polygon.')

There are 24 countries and 511 stations within the polygon.


In [36]:
p_cntr_element = pd.pivot_table(data=stations,
                                index='country',
                                columns='element',
                                values=['firstyear', 'lastyear'],
                                aggfunc={'firstyear': 'min', 'lastyear': 'max'})

In [37]:
p_cntr_element

firstyear                                 lastyear                  \
element      AWND    EVAP    MXPN    TMAX    TMIN     AWND    EVAP    MXPN   
country                                                                      
AC            NaN     NaN     NaN  1961.0  1961.0      NaN     NaN     NaN   
BB            NaN     NaN     NaN  1944.0  1944.0      NaN     NaN     NaN   
BD         1986.0     NaN     NaN  1945.0  1945.0   1995.0     NaN     NaN   
BF            NaN     NaN     NaN  1954.0  1954.0      NaN     NaN     NaN   
BR            NaN     NaN     NaN  1974.0  1973.0      NaN     NaN     NaN   
CA            NaN     NaN     NaN  1874.0  1874.0      NaN     NaN     NaN   
CV            NaN     NaN     NaN  1973.0  1973.0      NaN     NaN     NaN   
DO            NaN     NaN     NaN  1971.0  1971.0      NaN     NaN     NaN   
DR            NaN     NaN     NaN  1943.0  1943.0      NaN     NaN     NaN   
FG            NaN     NaN     NaN  1950.0  1950.0      NaN     NaN     NaN   
GP            NaN     NaN     NaN  1951.0  1951.0      NaN     NaN     NaN   
GY            NaN     NaN     NaN  1949.0  1949.0      NaN     NaN     NaN   
MB            NaN     NaN     NaN  1958.0  1958.0      NaN     NaN     NaN   
NN            NaN     NaN     NaN  1976.0  1982.0      NaN     NaN     NaN   
NS            NaN     NaN     NaN  1973.0  1973.0      NaN     NaN     NaN   
PO            NaN     NaN     NaN  1949.0  1949.0      NaN     NaN     NaN   
RQ         1984.0  1908.0  1963.0  1898.0  1898.0   2020.0  2020.0  2018.0   
SP            NaN     NaN     NaN  1919.0  1919.0      NaN     NaN     NaN   
ST            NaN     NaN     NaN  1948.0  1948.0      NaN     NaN     NaN   
TD            NaN     NaN     NaN  1945.0  1945.0      NaN     NaN     NaN   
UC            NaN     NaN     NaN  1973.0  1973.0      NaN     NaN     NaN   
US         1984.0     NaN     NaN  1893.0  1893.0   2020.0     NaN     NaN   
VE            NaN     NaN     NaN  1959.0  1959.0      NaN     NaN     NaN   
VQ         1994.0  1972.0  1972.0  1948.0  1948.0   2020.0  2014.0  2014.0   

                         
element    TMAX    TMIN  
country                  
AC       1961.0  1961.0  
BB       2018.0  2018.0  
BD       2020.0  2020.0  
BF       1960.0  1960.0  
BR       1974.0  1975.0  
CA       2020.0  2020.0  
CV       2020.0  2020.0  
DO       1973.0  1973.0  
DR       2020.0  2020.0  
FG       2020.0  2020.0  
GP       2018.0  2018.0  
GY       2020.0  2020.0  
MB       2018.0  2018.0  
NN       2008.0  2008.0  
NS       2014.0  2013.0  
PO       2020.0  2020.0  
RQ       2020.0  2020.0  
SP       2020.0  2020.0  
ST       1957.0  1957.0  
TD       2020.0  2020.0  
UC       2008.0  2008.0  
US       2020.0  2020.0  
VE       2018.0  2018.0  
VQ       2020.0  2020.0

It looks like the most reliable data is `TMIN` and `TMAX`. In addition, it seems like the stations in Puerto Rico (RQ) and The United States Virgin Islands (VQ) contain a full set of data for all required elements. We may try to incorporate those anyways in our analysis.

In [44]:
min_last_year = 2015
max_first_year = 1960

cntr_to_exclude = p_cntr_element[(p_cntr_element[('lastyear', 'TMIN')] < min_last_year)
                                 | (p_cntr_element[('firstyear', 'TMIN')] > max_first_year)].index.tolist()

print(f'We will exclude stations from the following countries: {(", ").join(cntr_to_exclude)} because of incomplete data.')

We will exclude stations from the following countries: AC, BF, BR, CV, DO, NN, NS, ST, UC because of incomplete data.


In [52]:
sel_stations = stations[~stations.country.isin(cntr_to_exclude)].copy()
print(f'{n_stations - len(sel_stations.id.unique())} stations have been discarded due to insufficient data.')

20 stations have been discarded due to insufficient data.


In [55]:
unq_sel_stations = sel_stations[['id', 'latitude', 'longitude']].drop_duplicates().copy()

In [64]:
polygon = {"type": "Polygon",
           "coordinates":[[[-12.950869, 53.43105],
                           [-10.342842, 35.002107],
                           [-19.931662, 13.986127],
                           [-9.352620, 0.0],
                           [-49.101915, 0],
                           [-71.903541, 13.516013],
                           [-76.957330, 34.883146]]]}

In [65]:
widget = KeplerGl(height=480, data={'stations': unq_sel_stations, 'poly': polygon})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [66]:
widget

KeplerGl(data={'stations':               id  latitude  longitude
2    BB000078954   13.0670   -59.4830
4    BD…